# Data Exploration

### Parse Files

In [9]:
import json

FILES = [
    'leagues.json',
    'player.json',
    'teams.json',
    'tournaments.json',
]

def parse(filename):
    f = open(filename)
    data = json.load(f)
    f.close()
    return data

leagues = parse('data/leagues.json')
players = parse('data/player.json')
teams = parse('data/teams.json')
tournaments = parse('data/tournaments.json')

### Pandas

In [17]:
import pandas
pandas.__version__

'2.1.1'

In [20]:
leagues = pandas.DataFrame(leagues)
players = pandas.DataFrame(players)
teams = pandas.DataFrame(teams)
tournaments = pandas.DataFrame(tournaments)

In [14]:
leagues

leagues : 38


AttributeError: 'list' object has no attribute 'keys'